## Churn Prediction Feature Engineering

<img src="https://github.com/RafiKurlansik/laughing-garbanzo/blob/main/step1.png?raw=true">

### Featurization Logic

This is a fairly clean dataset so we'll just do some one-hot encoding, and clean up the column names afterward.

In [0]:
# Read into Spark
telcoDF = spark.table("sr_ibm_telco_churn.bronze_customers")

display(telcoDF)

Using `koalas` allows us to scale `pandas` code.

In [0]:
from databricks.feature_store import feature_table
import databricks.koalas as ks

def compute_churn_features(data):
  
  # Convert to koalas
  data = data.to_koalas()
  
  # OHE
  data = ks.get_dummies(data, 
                        columns=['gender', 'partner', 'dependents',
                                 'phoneService', 'multipleLines', 'internetService',
                                 'onlineSecurity', 'onlineBackup', 'deviceProtection',
                                 'techSupport', 'streamingTV', 'streamingMovies',
                                 'contract', 'paperlessBilling', 'paymentMethod'],dtype = 'int64')
  
  # Convert label to int and rename column
  data['churnString'] = data['churnString'].map({'Yes': 1, 'No': 0})
  data = data.astype({'churnString': 'int32'})
  data = data.rename(columns = {'churnString': 'churn'})
  
  # Clean up column names
  data.columns = data.columns.str.replace(' ', '')
  data.columns = data.columns.str.replace('(', '-')
  data.columns = data.columns.str.replace(')', '')
  
  # Drop missing values
  data = data.dropna()
  
  return data

In [0]:
from databricks.feature_store import FeatureStoreClient

fs = FeatureStoreClient()

churn_features_df = compute_churn_features(telcoDF)

churn_feature_table = fs.create_feature_table(
  name='sr_ibm_telco_churn.churn_features',
  keys='customerID',
  schema=churn_features_df.spark.schema(),
  description='These features are derived from the sr_ibm_telco_churn.bronze_customers table in the lakehouse.  I created dummy variables for the categorical columns, cleaned up their names, and added a boolean flag for whether the customer churned or not.  No aggregations were performed.'
)

fs.write_table(df=churn_features_df.to_spark(), name='sr_ibm_telco_churn.churn_features', mode='overwrite')